### ------------------------------------
## Introduction
### ------------------------------------

### Greenspot Grocer - Data Cleaning & Normalization

This notebook takes the raw Greenspot Grocer inventory/sales spreadsheet 
and performs:
- Data exploration
- Missing value filling
- Data normalization into multiple CSVs for MySQL import
- Preparation for relational database creation

Author: Samuel Akuffo

Date: 2025-03-08

In [ ]:
import pandas as pd
import numpy as np
from datetime import date

# Display settings for easier viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [ ]:
# Loading the raw data to clean

raw_data = "GreenspotDataset.csv"

df = pd.read_csv(raw_data)

# Removing any row where all columns are NaN or empty strings
df = df.replace(r'^\s*$', pd.NA, regex=True) 
df = df.dropna(how="all").reset_index(drop=True)

print("Clean dataset shape:", df.shape)
df

Clean dataset shape: (21, 13)


,Item num,description,quantity on-hand,cost,purchase date,vendor,price,date sold,cust,Quantity,item type,Location,Unit
0,1000.0,Bennet Farm free-range eggs,29.0,2.35,2/1/2022,"Bennet Farms, Rt. 17 Evansville, IL 55446",NaN,NaN,NaN,25.0,Dairy,D12,dozen
1,1000.0,Bennet Farm free-range eggs,27.0,NaN,NaN,NaN,5.49,2/2/2022,198765,2.0,Dairy,D12,dozen
2,2000.0,Ruby's Kale,3.0,NaN,NaN,NaN,3.99,2/2/2022,<NA>,2.0,Produce,p12,bunch
3,1100.0,Freshness White beans,13.0,NaN,NaN,NaN,1.49,2/2/2022,202900,2.0,Canned,a2,12 ounce can
4,1100.0,Freshness White beans,53.0,0.69,2/2/2022,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",NaN,NaN,NaN,40.0,Canned,a2,12 oz can
5,1000.0,Bennet Farm free-range eggs,25.0,NaN,NaN,NaN,5.99,2/4/2022,196777,2.0,Dairy,D12,dozen
6,1100.0,Freshness White beans,45.0,NaN,NaN,NaN,1.49,2/7/2022,198765,8.0,Canned,a2,12-oz can
7,1222.0,Freshness Green beans,59.0,0.59,2/10/2022,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",NaN,NaN,NaN,40.0,Canned,a3,12 ounce can
8,1223.0,Freshness Green beans,12.0,1.75,2/10/2022,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",NaN,NaN,NaN,10.0,Canned,a7,36 oz can
9,1224.0,Freshness Wax beans,31.0,0.65,2/10/2022,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",NaN,NaN,NaN,30.0,Canned,a3,12 ounce can


## Initial Data Load and Observations from Raw Dataset

Upon loading the raw Greenspot Grocery transaction dataset (`GreenspotDataset.csv`), a range of structural, semantic, and formatting issues were identified. These issues significantly affect data usability, accuracy, and scalability. Below are the main data quality concerns:

---

### 1. Non-Linear and Incomplete Time Series
- The dataset lacks early purchase or inventory history.
- Sales appear before any matching stock entry in some cases.
- As a result, time-based analyses like forecasting or inventory valuation would be inaccurate.

---

### 2. Dual-Purpose Columns with Poor Labeling
- `quantity on-hand` refers to stock remaining.
- `Quantity` refers to the number of units sold.
- Without renaming or contextual grouping, these columns are ambiguous and error-prone.

---

### 3. Missing and Inferred Cost/Price Data
- Only the first row for each product includes `cost` or `price`.
- All subsequent transactions for that product have missing values.
- This leads to:
  - Manual inference.
  - Risk of using outdated pricing.
  - Complications when cost varies by vendor or time.

---

### 4. Split Timeline Columns
- Uses both `purchase date` and `date sold`:
  - One is always empty depending on the transaction type.
- Leads to unnecessary nulls and complicates date tracking.
- Solution: unified `transaction_date` + a `transaction_type` field.

---

### 5. Vendor Field Formatting Issues
- Vendor name and full address are packed into a single string.
- Makes it difficult to:
  - Normalize data.
  - Query by state or ZIP.
  - Enforce location integrity.

---

### 6. Sparse and Incomplete Customer Data
- `cust` only contains a numeric ID (many of which are missing).
- No name, contact info, or metadata.
- Sales rows with missing `cust` values can't be reliably attributed.

---

### 7. Inconsistent Values in Key Fields
- Variants like `D12`, `d12 `, and ` D12` all refer to the same location.
- Units such as `12 ounce can`, `12 oz can`, and `12-oz can` are treated as separate values.
- Causes:
  - Data redundancy.
  - Poor grouping in reports.
  - Complicated joins in SQL.

---

## Key Takeaway

The original dataset suffers from poor formatting, missing values, lack of naming standards, and ambiguous relationships. Left uncorrected, these issues would hinder inventory tracking, sales reporting, and system automation. A full normalization process, along with standardization and schema design, was necessary to convert this dataset into a structured, reliable foundation for the business.

In [ ]:
# Exploring the data and checking for missing values per column

print("\nMissing values per column: ")
for i, entity in enumerate(df, start=1):
    entity_sum = df[entity].isnull().sum()
    print(f"{i}. There are {entity_sum} values missing in the {entity} column")

print(f"\nThere are {df.isnull().sum().sum()} missing values in total")


Missing values per column: 
1. There are 0 values missing in the Item num column
2. There are 0 values missing in the description column
3. There are 0 values missing in the quantity on-hand column
4. There are 13 values missing in the cost column
5. There are 13 values missing in the purchase date column
6. There are 13 values missing in the vendor column
7. There are 8 values missing in the price column
8. There are 8 values missing in the date sold column
9. There are 11 values missing in the cust column
10. There are 0 values missing in the Quantity column
11. There are 0 values missing in the item type column
12. There are 0 values missing in the Location column
13. There are 0 values missing in the Unit column

There are 66 missing values in total


In [ ]:
# Checking unique product descriptions to understand how to impute missing values
descriptions = sorted(df['description'].dropna().unique())

print("The unique descriptions in the dataset are: ")

for i, unique_description in enumerate(descriptions, start=1):
    count = df[df['description'] == unique_description].shape[0]
    print(f"{i}. {unique_description} (appears {count} times).")

The unique descriptions in the dataset are: 
1. Bennet Farm free-range eggs (appears 4 times).
2. Freshness Green beans (appears 5 times).
3. Freshness Wax beans (appears 2 times).
4. Freshness White beans (appears 4 times).
5. Ruby's Kale (appears 3 times).
6. Ruby's Organic Kale (appears 3 times).


In [ ]:
# Checking and displaying the unique item types along with their frequencies
item_type = sorted(df['item type'].dropna().unique())

print("The unique item types in the dataset are: ")

for i, unique_item in enumerate(item_type, start=1):
    count = df[df['item type'] == unique_item].shape[0]
    print(f"{i}. {unique_item} (appears {count} times).")

The unique item types in the dataset are: 
1. Canned (appears 11 times).
2. Dairy (appears 4 times).
3. Produce (appears 6 times).


In [ ]:
# Replacing missing vendor values by inferring from product descriptions

vendor_mapping = {
    "Bennet": "Bennet Farms, Rt. 17 Evansville, IL 55446",
    "Ruby's": "Ruby Redd Produce, LLC, 1212 Milam St., Kenosha, AL, 34567",
    "Freshness": "Freshness, Inc., 202 E. Maple St., St. Joseph, MO 45678",
}

for keyword, vendor_value in vendor_mapping.items():
    df.loc[
        df['description'].str.contains(keyword, case=False, na=False), 'vendor'
    ] = vendor_value

print("\nVendor column after filling:")
print(df['vendor'].value_counts())


Vendor column after filling:
vendor
Freshness, Inc., 202 E. Maple St., St. Joseph, MO 45678       11
Ruby Redd Produce, LLC, 1212 Milam St., Kenosha, AL, 34567     6
Bennet Farms, Rt. 17 Evansville, IL 55446                      4
Name: count, dtype: int64


# Data Standardization and Type Conversion

- Converted date columns (`purchase date`, `date sold`) from string to date objects to enable date operations and accurate storage in the database.
- Standardized unit descriptions by replacing variations (e.g., "12 ounce can" and "12-oz can") with a consistent label ("12 oz can") for data consistency.
- Converted numeric columns (`cost`, `price`, `cust`, `Quantity`, `quantity on-hand`, and `Item num`) to proper numeric types to ensure accurate calculations and database compatibility.
- Further ensured integer fields are stored with integer data types (`Int64`) to avoid unintended floating-point representation.

In [ ]:
# Data Standardization: Dates, Units, Cost and Price

# Enforcing date format to date from string
date_cols = [
    'purchase date', 'date sold'
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

# Standardizing units which had variations
df['Unit'] = df['Unit'].str.replace("12 ounce can | 12-oz can", "12 oz can", case=False, regex=True)

# Ensuring numerals in the cost and price columns are numbers
df['cost'] = pd.to_numeric(df['cost'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['cust'] = pd.to_numeric(df['cust'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['quantity on-hand'] = pd.to_numeric(df['quantity on-hand'], errors='coerce')
df['Item num'] = pd.to_numeric(df['Item num'], errors='coerce')

# Changing numbers that should not be floats to non floatable numbers
df['Quantity'] = df['Quantity'].astype('Int64')
df['quantity on-hand'] = df['quantity on-hand'].astype('Int64')
df['Item num'] = df['Item num'].astype('Int64')
df['cust'] = df['cust'].astype('Int64')

# Filling missing cost and price values by inferring from other rows with the same product description

Some rows have missing `cost` or `price` values even though the same product’s cost or price is available in other rows.  
To address this, I create lookup tables grouped by product description and fill missing values accordingly.

In [ ]:
# Creating lookup tables for cost and price by product description

cost_lookup = df.groupby('description')['cost'].first()
price_lookup = df.groupby('description')['price'].first()

# Fill missing cost values using the lookup
df['cost'] = df.apply (
    lambda row: cost_lookup[row['description']] if pd.isna(row['cost']) else row['price'], axis=1
)

# Fill missing price values using the lookup
df['price'] = df.apply (
    lambda row: price_lookup[row['description']] if pd.isna(row['price']) else row['price'], axis=1
)

df

,Item num,description,quantity on-hand,cost,purchase date,vendor,price,date sold,cust,Quantity,item type,Location,Unit
0,1000,Bennet Farm free-range eggs,29,2.35,2022-02-01,"Bennet Farms, Rt. 17 Evansville, IL 55446",5.49,NaT,<NA>,25,Dairy,D12,dozen
1,1000,Bennet Farm free-range eggs,27,5.49,NaT,"Bennet Farms, Rt. 17 Evansville, IL 55446",5.49,2022-02-02,198765,2,Dairy,D12,dozen
2,2000,Ruby's Kale,3,3.99,NaT,"Ruby Redd Produce, LLC, 1212 Milam St., Kenosh...",3.99,2022-02-02,<NA>,2,Produce,p12,bunch
3,1100,Freshness White beans,13,1.49,NaT,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",1.49,2022-02-02,202900,2,Canned,a2,12 ounce can
4,1100,Freshness White beans,53,0.69,2022-02-02,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",1.49,NaT,<NA>,40,Canned,a2,12 oz can
5,1000,Bennet Farm free-range eggs,25,5.99,NaT,"Bennet Farms, Rt. 17 Evansville, IL 55446",5.99,2022-02-04,196777,2,Dairy,D12,dozen
6,1100,Freshness White beans,45,1.49,NaT,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",1.49,2022-02-07,198765,8,Canned,a2,12-oz can
7,1222,Freshness Green beans,59,0.59,2022-02-10,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",1.29,NaT,<NA>,40,Canned,a3,12 ounce can
8,1223,Freshness Green beans,12,0.59,2022-02-10,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",1.29,NaT,<NA>,10,Canned,a7,36 oz can
9,1224,Freshness Wax beans,31,0.65,2022-02-10,"Freshness, Inc., 202 E. Maple St., St. Joseph,...",1.55,NaT,<NA>,30,Canned,a3,12 ounce can


# Vendor Data Normalization

The `vendor` column contains multiple pieces of information combined into a single string (vendor name, street, city, state, and zipcode). To support database normalization, I created a function `vendor_parse` that parses and splits this string into individual components:

- Handles different vendor string formats (e.g., varying number of commas or formatting styles).
- Returns a list of normalized fields: vendor name, street, city, state, and zipcode.
- Applies this function to the entire dataset and creates separate columns for each component.

This approach allows easier querying and ensures consistent vendor data storage.

In [ ]:
# Changing vendor from a multivalued attribute to support normalization

def vendor_parse(vendor_str):
    if not vendor_str or pd.isna(vendor_str):
        return [None] * 5

    parts = [p.strip() for p in vendor_str.split(',')]

    # Case 1: Ruby Redd style → 6 parts
    if len(parts) == 6:
        vendor_name = parts[0] + ', ' + parts[1]  # join name and LLC
        vendor_street = parts[2]
        vendor_city = parts[3]
        vendor_state = parts[4]
        vendor_zipcode = parts[5]

    # Case 2: Freshness style → 5 parts, state & zip are together
    elif len(parts) == 5:
        vendor_name = parts[0] + ', ' + parts[1]
        vendor_street = parts[2]
        vendor_city = parts[3]
        state_zip = parts[4].split()
        vendor_state = state_zip[0]
        vendor_zipcode = state_zip[1]

    # Case 3: Bennet style → 3 parts, street+city together
    elif len(parts) == 3:
        vendor_name = parts[0]
        street_city = parts[1].rsplit(' ', 1)
        vendor_street = street_city[0]
        vendor_city = street_city[1]
        state_zip = parts[2].split()
        vendor_state = state_zip[0]
        vendor_zipcode = state_zip[1]

    else:
        # Fallback for unknown patterns
        return [vendor_str, None, None, None, None]

    return [vendor_name, vendor_street, vendor_city, vendor_state, vendor_zipcode]


# Test
sample_vendors = [
    "Ruby Redd Produce, LLC, 1212 Milam St., Kenosha, AL, 34567",
    "Freshness, Inc., 202 E. Maple St., St. Joseph, MO 45678",
    "Bennet Farms, Rt. 17 Evansville, IL 55446"
]

for vendor in sample_vendors:
    parsed = vendor_parse(vendor)
    print(f"Original: {vendor}")
    print(f"Parsed: {parsed}")
    print("------")

vendor_cols = df['vendor'].apply(vendor_parse)
df[['vendor_name','vendor_street', 'vendor_city', 'vendor_state', 'vendor_zipcode']] = pd.DataFrame(vendor_cols.tolist(), index=df.index)

Original: Ruby Redd Produce, LLC, 1212 Milam St., Kenosha, AL, 34567
Parsed: ['Ruby Redd Produce, LLC', '1212 Milam St.', 'Kenosha', 'AL', '34567']
------
Original: Freshness, Inc., 202 E. Maple St., St. Joseph, MO 45678
Parsed: ['Freshness, Inc.', '202 E. Maple St.', 'St. Joseph', 'MO', '45678']
------
Original: Bennet Farms, Rt. 17 Evansville, IL 55446
Parsed: ['Bennet Farms', 'Rt. 17', 'Evansville', 'IL', '55446']
------


In [11]:
df = df.drop("vendor", axis=1)
df

,Item num,description,quantity on-hand,cost,purchase date,price,date sold,cust,Quantity,item type,Location,Unit,vendor_name,vendor_street,vendor_city,vendor_state,vendor_zipcode
0,1000,Bennet Farm free-range eggs,29,2.35,2022-02-01,5.49,NaT,<NA>,25,Dairy,D12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
1,1000,Bennet Farm free-range eggs,27,5.49,NaT,5.49,2022-02-02,198765,2,Dairy,D12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
2,2000,Ruby's Kale,3,3.99,NaT,3.99,2022-02-02,<NA>,2,Produce,p12,bunch,"Ruby Redd Produce, LLC",1212 Milam St.,Kenosha,AL,34567
3,1100,Freshness White beans,13,1.49,NaT,1.49,2022-02-02,202900,2,Canned,a2,12 ounce can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
4,1100,Freshness White beans,53,0.69,2022-02-02,1.49,NaT,<NA>,40,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
5,1000,Bennet Farm free-range eggs,25,5.99,NaT,5.99,2022-02-04,196777,2,Dairy,D12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
6,1100,Freshness White beans,45,1.49,NaT,1.49,2022-02-07,198765,8,Canned,a2,12-oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
7,1222,Freshness Green beans,59,0.59,2022-02-10,1.29,NaT,<NA>,40,Canned,a3,12 ounce can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
8,1223,Freshness Green beans,12,0.59,2022-02-10,1.29,NaT,<NA>,10,Canned,a7,36 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
9,1224,Freshness Wax beans,31,0.65,2022-02-10,1.55,NaT,<NA>,30,Canned,a3,12 ounce can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678


# Determining Transaction Types and Dates

Since the original dataset records purchase and sale dates in separate columns (`purchase date` and `date sold`), it's necessary to consolidate these to support consistent transaction tracking.

- The `transaction_type` column is created to label each record as either a "SALE" or "PURCHASE" based on whether the `date sold` field is present.
- The `transaction_date` column consolidates the relevant date by prioritizing the `date sold` if present; otherwise, it uses the `purchase date`.

This transformation simplifies subsequent data processing by combining sales and purchases into a unified transaction timeline.

In [ ]:
# Assigning transaction type: 'SALE' if 'date sold' exists, else 'PURCHASE'
df['transaction_type'] = df.apply(
    lambda row: 'SALE' if pd.notna(row['date sold']) else 'PURCHASE', axis=1
)

# Creating unified transaction date, preferring 'date sold' over 'purchase date'
df['transaction_date'] = df['date sold'].combine_first(df['purchase date'])

# Standardizing Unit Names

The `Unit` column contains variations of the same unit descriptions, such as "12 ounce can" and "12-oz can." 

To maintain consistency and improve data quality, these variants are standardized to a single form: "12 oz can."

In [ ]:
# Replace different variants of "12 ounce can" with standardized "12 oz can"
df['Unit'] = df['Unit'].str.replace("12 ounce can", "12 oz can", case=False)

# Replace "12-oz can" variant with the standardized "12 oz can"
df['Unit'] = df['Unit'].str.replace("12-oz can", "12 oz can", case=False)

In [17]:
df = df.drop(columns=['date sold', 'purchase date'])

df

,Item num,description,quantity on-hand,cost,price,cust,Quantity,item type,Location,Unit,vendor_name,vendor_street,vendor_city,vendor_state,vendor_zipcode,transaction_type,transaction_date
0,1000,Bennet Farm free-range eggs,29,2.35,5.49,<NA>,25,Dairy,D12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446,PURCHASE,2022-02-01
1,1000,Bennet Farm free-range eggs,27,5.49,5.49,198765,2,Dairy,D12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446,SALE,2022-02-02
2,2000,Ruby's Kale,3,3.99,3.99,<NA>,2,Produce,p12,bunch,"Ruby Redd Produce, LLC",1212 Milam St.,Kenosha,AL,34567,SALE,2022-02-02
3,1100,Freshness White beans,13,1.49,1.49,202900,2,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678,SALE,2022-02-02
4,1100,Freshness White beans,53,0.69,1.49,<NA>,40,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678,PURCHASE,2022-02-02
5,1000,Bennet Farm free-range eggs,25,5.99,5.99,196777,2,Dairy,D12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446,SALE,2022-02-04
6,1100,Freshness White beans,45,1.49,1.49,198765,8,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678,SALE,2022-02-07
7,1222,Freshness Green beans,59,0.59,1.29,<NA>,40,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678,PURCHASE,2022-02-10
8,1223,Freshness Green beans,12,0.59,1.29,<NA>,10,Canned,a7,36 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678,PURCHASE,2022-02-10
9,1224,Freshness Wax beans,31,0.65,1.55,<NA>,30,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678,PURCHASE,2022-02-10


# Renaming Columns and Reordering DataFrame

To align the dataset with the database schema and improve readability, I renamed columns to more descriptive and consistent names. 

After renaming, I reordered the columns to group related data logically and for easier reference during database insertion and analysis.

In [ ]:
# Rename columns to match database schema and improve clarity
df = df.rename(columns={
    'Item num': 'product_item_id',
    'description': 'product_description',
    'quantity on-hand': 'stock_quantity',
    'cost': 'wholesale_cost',
    'purchase date': 'purchase_date',
    'vendor': 'vendor',
    'price': 'sale_price',
    'cust': 'customer_id',
    'Quantity': 'quantity_changed',
    'item type': 'item_type',
    'location': 'location_name',
    'Unit': 'unit',
})

# Reorder columns to logically group fields for easier handling
df = df[['product_item_id', 'product_description', 'stock_quantity', 'quantity_changed', 'transaction_type', 'wholesale_cost', 
               'transaction_date','sale_price','customer_id',  'item_type', 'location_name', 'unit',
               'vendor_name', 'vendor_street', 'vendor_city', 'vendor_state', 'vendor_zipcode']]

df

,product_item_id,product_description,stock_quantity,quantity_changed,transaction_type,wholesale_cost,transaction_date,sale_price,customer_id,item_type,location_name,unit,vendor_name,vendor_street,vendor_city,vendor_state,vendor_zipcode
0,1000,Bennet Farm free-range eggs,29,25,PURCHASE,2.35,2022-02-01,5.49,<NA>,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
1,1000,Bennet Farm free-range eggs,27,-2,SALE,2.35,2022-02-02,5.49,198765,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
5,1000,Bennet Farm free-range eggs,25,-2,SALE,2.35,2022-02-04,5.49,196777,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
10,1000,Bennet Farm free-range eggs,21,-4,SALE,2.35,2022-02-11,5.49,277177,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
3,1100,Freshness White beans,13,-2,SALE,0.69,2022-02-02,1.49,202900,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
4,1100,Freshness White beans,53,40,PURCHASE,0.69,2022-02-02,1.49,<NA>,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
6,1100,Freshness White beans,45,-8,SALE,0.69,2022-02-07,1.49,198765,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
11,1100,Freshness White beans,41,-4,SALE,0.69,2022-02-11,1.49,1277177,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
7,1222,Freshness Green beans,59,40,PURCHASE,0.59,2022-02-10,1.29,<NA>,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
12,1222,Freshness Green beans,47,-12,SALE,0.59,2022-02-12,1.29,111000,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678


# Cleaning and Sorting Data

I standardized the `location_name` column by trimming whitespace and converting all values to lowercase to avoid inconsistencies due to case or trailing spaces.

Then, I sorted the DataFrame by `product_description` and `transaction_date` to organize the data chronologically within each product for easier analysis.

In [ ]:
# Standardize location names: remove leading/trailing spaces and convert to lowercase
df = df['location_name'] = df['location_name'].str.strip().str.lower()

# Sort data by product description and transaction date for chronological order within products
df = df.sort_values(by=['product_description', 'transaction_date'], ascending=[True, True])

# Standardizing Product Costs and Sale Prices

To ensure consistency in pricing data, I mapped known wholesale costs and sale prices to each product based on their `product_item_id`. 

If any cost or price was missing or incorrect in the original data, this mapping fills those gaps with the correct values.

In [ ]:
# Define fixed wholesale cost per product item ID
cost_map = {
    1000 : 2.35,
    1100 : 0.69,
    1222 : 0.59,
    1223 : 1.75,
    1224 : 0.65,
    2000 : 1.29,
    2001 : 2.19,
}

# Define fixed sale price per product item ID
sale_map = {
    1000 : 5.49,
    1100 : 1.49,
    1222 : 1.29,
    1223 : 3.49,
    1224 : 1.55,
    2000 : 3.99,
    2001 : 6.99,
}

# Map and fill wholesale_cost and sale_price based on product_item_id
df['wholesale_cost'] = df['product_item_id'].map(cost_map).fillna(df['wholesale_cost'])
df['sale_price'] = df['product_item_id'].map(sale_map).fillna(df['sale_price'])

# Adjusting Quantity Changes for Transaction Types

For sale transactions, the quantity change represents inventory going out, so the quantity values are made negative. For purchase transactions, quantities remain positive to indicate stock added.

In [ ]:
# Negate quantity_changed for sales to reflect stock decrease

df['quantity_changed'] = df.apply(
    lambda row: -row['quantity_changed'] if row['transaction_type'] == 'SALE' else row['quantity_changed'],
    axis=1
)

In [50]:
df = df.sort_values(['product_item_id', 'transaction_date'])
df

,product_item_id,product_description,stock_quantity,quantity_changed,transaction_type,wholesale_cost,transaction_date,sale_price,customer_id,item_type,location_name,unit,vendor_name,vendor_street,vendor_city,vendor_state,vendor_zipcode
0,1000,Bennet Farm free-range eggs,29,25,PURCHASE,2.35,2022-02-01,5.49,<NA>,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
1,1000,Bennet Farm free-range eggs,27,2,SALE,2.35,2022-02-02,5.49,198765,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
5,1000,Bennet Farm free-range eggs,25,2,SALE,2.35,2022-02-04,5.49,196777,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
10,1000,Bennet Farm free-range eggs,21,4,SALE,2.35,2022-02-11,5.49,277177,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
3,1100,Freshness White beans,13,2,SALE,0.69,2022-02-02,1.49,202900,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
4,1100,Freshness White beans,53,40,PURCHASE,0.69,2022-02-02,1.49,<NA>,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
6,1100,Freshness White beans,45,8,SALE,0.69,2022-02-07,1.49,198765,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
11,1100,Freshness White beans,41,4,SALE,0.69,2022-02-11,1.49,1277177,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
7,1222,Freshness Green beans,59,40,PURCHASE,0.59,2022-02-10,1.29,<NA>,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
12,1222,Freshness Green beans,47,12,SALE,0.59,2022-02-12,1.29,111000,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678


In [51]:
filtered_rows = df[df['product_item_id'] == 1100]
filtered_rows[['product_item_id', 'product_description','transaction_date',	'customer_id','stock_quantity', 'quantity_changed',	'transaction_type']].sort_values(by='transaction_date')


,product_item_id,product_description,transaction_date,customer_id,stock_quantity,quantity_changed,transaction_type
3,1100,Freshness White beans,2022-02-02,202900,13,2,SALE
4,1100,Freshness White beans,2022-02-02,<NA>,53,40,PURCHASE
6,1100,Freshness White beans,2022-02-07,198765,45,8,SALE
11,1100,Freshness White beans,2022-02-11,1277177,41,4,SALE


In [52]:
sale_df = df[df['transaction_type'] == 'SALE']
sale_df[['product_item_id', 'product_description','transaction_type', 'transaction_date',	'customer_id','stock_quantity', 'quantity_changed']]

,product_item_id,product_description,transaction_type,transaction_date,customer_id,stock_quantity,quantity_changed
1,1000,Bennet Farm free-range eggs,SALE,2022-02-02,198765,27,2
5,1000,Bennet Farm free-range eggs,SALE,2022-02-04,196777,25,2
10,1000,Bennet Farm free-range eggs,SALE,2022-02-11,277177,21,4
3,1100,Freshness White beans,SALE,2022-02-02,202900,13,2
6,1100,Freshness White beans,SALE,2022-02-07,198765,45,8
11,1100,Freshness White beans,SALE,2022-02-11,1277177,41,4
12,1222,Freshness Green beans,SALE,2022-02-12,111000,47,12
16,1223,Freshness Green beans,SALE,2022-02-13,198765,7,5
13,1224,Freshness Wax beans,SALE,2022-02-12,111000,23,8
2,2000,Ruby's Kale,SALE,2022-02-02,1277177,3,2


# Correcting Missing Customer IDs

Here, I manually assign missing `customer_id` values for specific product sales on particular transaction dates, based on available data and inferred information. This ensures consistency and completeness in customer identification.

In [ ]:
# Filling missing customer_id values for specific product sales on given dates

df.loc[
    (df['product_description'] == 'Freshness Wax beans') &
    (df['transaction_date'] == date(2022, 2, 12)),
    'customer_id'
] = 111000

df.loc[
    (df['product_description'] == 'Freshness White beans') &
    (df['transaction_date'] == date(2022, 2, 11)),
    'customer_id'
] = 1277177

df.loc[
    (df['product_description'] == "Ruby's Kale") &
    (df['transaction_date'] == date(2022, 2, 2)),
    'customer_id'
] = 1277177

df

,product_item_id,product_description,stock_quantity,quantity_changed,transaction_type,wholesale_cost,transaction_date,sale_price,customer_id,item_type,location_name,unit,vendor_name,vendor_street,vendor_city,vendor_state,vendor_zipcode
0,1000,Bennet Farm free-range eggs,29,25,PURCHASE,2.35,2022-02-01,5.49,<NA>,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
1,1000,Bennet Farm free-range eggs,27,2,SALE,2.35,2022-02-02,5.49,198765,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
5,1000,Bennet Farm free-range eggs,25,2,SALE,2.35,2022-02-04,5.49,196777,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
10,1000,Bennet Farm free-range eggs,21,4,SALE,2.35,2022-02-11,5.49,277177,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
3,1100,Freshness White beans,13,2,SALE,0.69,2022-02-02,1.49,202900,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
4,1100,Freshness White beans,53,40,PURCHASE,0.69,2022-02-02,1.49,<NA>,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
6,1100,Freshness White beans,45,8,SALE,0.69,2022-02-07,1.49,198765,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
11,1100,Freshness White beans,41,4,SALE,0.69,2022-02-11,1.49,1277177,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
7,1222,Freshness Green beans,59,40,PURCHASE,0.59,2022-02-10,1.29,<NA>,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
12,1222,Freshness Green beans,47,12,SALE,0.59,2022-02-12,1.29,111000,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678


In [68]:
df_cleaned = df.copy()

df_cleaned.reset_index(drop=True, inplace=True)

df_cleaned

,product_item_id,product_description,stock_quantity,quantity_changed,transaction_type,wholesale_cost,transaction_date,sale_price,customer_id,item_type,location_name,unit,vendor_name,vendor_street,vendor_city,vendor_state,vendor_zipcode
0,1000,Bennet Farm free-range eggs,29,25,PURCHASE,2.35,2022-02-01,5.49,<NA>,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
1,1000,Bennet Farm free-range eggs,27,-2,SALE,2.35,2022-02-02,5.49,198765,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
2,1000,Bennet Farm free-range eggs,25,-2,SALE,2.35,2022-02-04,5.49,196777,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
3,1000,Bennet Farm free-range eggs,21,-4,SALE,2.35,2022-02-11,5.49,277177,Dairy,d12,dozen,Bennet Farms,Rt. 17,Evansville,IL,55446
4,1100,Freshness White beans,13,-2,SALE,0.69,2022-02-02,1.49,202900,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
5,1100,Freshness White beans,53,40,PURCHASE,0.69,2022-02-02,1.49,<NA>,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
6,1100,Freshness White beans,45,-8,SALE,0.69,2022-02-07,1.49,198765,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
7,1100,Freshness White beans,41,-4,SALE,0.69,2022-02-11,1.49,1277177,Canned,a2,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
8,1222,Freshness Green beans,59,40,PURCHASE,0.59,2022-02-10,1.29,<NA>,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678
9,1222,Freshness Green beans,47,-12,SALE,0.59,2022-02-12,1.29,111000,Canned,a3,12 oz can,"Freshness, Inc.",202 E. Maple St.,St. Joseph,MO,45678


# Data Normalization into Relational Tables

Now that the flat transactional dataset is cleaned and standardized, we normalize it into separate dimension and fact tables for better structure, scalability, and use in a relational database or data warehouse.

The normalized tables are:
- `vendor_df`
- `unit_df`
- `item_type_df`
- `location_df`
- `customer_df`
- `product_df`
- `inventory_df`
- `inventory_transaction_df`
- `purchase_df`
- `sale_df`

In [ ]:
# Normalizing the flat DataFrame into the appropriate tables

# Vendor Table
vendor_cols = ['vendor_name', 'vendor_street', 'vendor_city', 'vendor_state', 'vendor_zipcode']
vendor_df = df_cleaned[vendor_cols].drop_duplicates().reset_index(drop=True)
vendor_df['vendor_id'] = vendor_df.index + 1000

# Unit Table
unit_df = df_cleaned[['unit']].drop_duplicates().reset_index(drop=True)
unit_df = unit_df.rename(columns={'unit': 'unit_name'})
unit_df['unit_id'] = unit_df.index + 100

# Item Type Table
item_type_df = df_cleaned[['item_type']].drop_duplicates().reset_index(drop=True)
item_type_df = item_type_df.rename(columns={'item_type': 'item_type_name'})
item_type_df['item_type_id'] = item_type_df.index +100

# Location Table
location_df = df_cleaned[['location_name']].drop_duplicates().reset_index(drop=True)
location_df = location_df.rename(columns={'location_name':'location_name'})

# Customer Table
customer_df = df_cleaned[df_cleaned['transaction_type'] == 'SALE'][['customer_id']].dropna().drop_duplicates().reset_index(drop=True)
customer_df['customer_id'] = customer_df['customer_id'].astype(int)

# Product Table
product_df = df_cleaned[['product_item_id', 'product_description', 'unit', 'item_type']].drop_duplicates()
product_df = product_df.merge(unit_df, left_on='unit', right_on='unit_name')
product_df = product_df.merge(item_type_df, left_on='item_type', right_on='item_type_name')
product_df = product_df[['product_item_id', 'product_description', 'unit_id', 'item_type_id']] 

# Inventory Table
inventory_df = df_cleaned[df_cleaned['transaction_type'] == 'PURCHASE'][['product_item_id', 'location_name']]
inventory_df = inventory_df[['product_item_id', 'location_name']].drop_duplicates().reset_index(drop=True)
inventory_df['inventory_id'] = inventory_df.index + 1000

# Inventory Transaction Table
inventory_transaction_df = df_cleaned.merge(inventory_df, on=['product_item_id', 'location_name'], how='left')
inventory_transaction_df = inventory_transaction_df[['inventory_id', 'transaction_date', 'transaction_type', 'stock_quantity', 'quantity_changed']].copy()
inventory_transaction_df = inventory_transaction_df.reset_index(drop=True)
inventory_transaction_df['transaction_id'] = inventory_transaction_df.index + 1000

# Purchase Table
purchase_df = df_cleaned[df_cleaned['transaction_type'] == 'PURCHASE'].reset_index(drop=True)
purchase_df = purchase_df.merge(vendor_df, on=vendor_cols)
purchase_df = purchase_df.merge(inventory_df[['product_item_id', 'location_name', 'inventory_id']], 
                                 on=['product_item_id', 'location_name'], how='left')
purchase_tx_ids = inventory_transaction_df[
    inventory_transaction_df['transaction_type'] == 'PURCHASE'
][['inventory_id', 'transaction_id']]
purchase_df = purchase_df.merge(purchase_tx_ids, on='inventory_id', how='left')
purchase_df = purchase_df[['product_item_id', 'vendor_id', 'wholesale_cost', 'transaction_id']].drop_duplicates().reset_index(drop=True)
purchase_df['purchase_id'] = purchase_df.index + 1000

# Sale Transaction Table
sale_df = df_cleaned[df_cleaned['transaction_type'] == 'SALE']
sale_df = sale_df.merge(inventory_transaction_df[['inventory_id', 'transaction_id', 'transaction_date']],left_index=True, right_index=True)
sale_df = sale_df[['product_item_id', 'customer_id','transaction_id', 'sale_price']].drop_duplicates().reset_index(drop=True)
sale_df['sale_id'] = sale_df.index + 1000


> Each resulting DataFrame now represents a normalized table that can be easily inserted into a relational schema for efficient storage, query performance, and referential integrity.

In [ ]:
# ------------------------------------
# 7. Save Normalized CSVs
# ------------------------------------

# Export all normalized DataFrames to CSV (no index)
vendor_df.to_csv("vendor.csv", index=False)
unit_df.to_csv("unit.csv", index=False)
item_type_df.to_csv("item_type.csv", index=False)
product_df.to_csv("product.csv", index=False)
location_df.to_csv("location.csv", index=False)
purchase_df.to_csv("purchase.csv", index=False)
inventory_df.to_csv("inventory.csv", index=False)
inventory_transaction_df.to_csv("inventory_transaction.csv", index=False)
customer_df.to_csv("customer.csv", index=False)
sale_df.to_csv("sales.csv", index=False)  # 🔧 corrected filename to lowercase for consistency

# Print confirmation
print("Normalized CSV files saved:")
print("- vendor.csv")
print("- unit.csv")
print("- item_type.csv")
print("- product.csv")
print("- location.csv")
print("- purchase.csv")
print("- inventory.csv")
print("- inventory_transaction.csv")
print("- customer.csv")
print("- sales.csv")

## Final Summary: Data Normalization & Cleaning for Greenspot Grocery

This notebook documents the complete process of cleaning, transforming, and normalizing the Greenspot Grocery transaction dataset in preparation for integration into a relational database (MySQL). The workflow reflects both automated and manual efforts to ensure high-quality, consistent, and normalized data outputs.

---

### 1. Initial Data Inspection & Cleaning
- Merged datasets from **purchase** and **sale** transactions.
- Identified and resolved column mismatches and structural inconsistencies.
- Standardized text fields:
  - `location_name` and `product_description` were stripped of whitespace and converted to lowercase for uniformity.

---

### 2. Transaction Preparation & Correction
- **Transaction sorting** was done by `product_description` and `transaction_date` to allow for sequential logic checks.
- Introduced a calculated field `quantity_changed`:
  - Negative for sales, positive for purchases, to reflect stock movements.
- Resolved temporal inconsistencies, such as **sales preceding purchases**, by ensuring transactions are ordered logically for inventory matching.

---

### 3. Manual Data Corrections
- **Manually assigned missing `customer_id`s** based on context (e.g., product, date, quantity).
- Verified and ensured **no duplicate transaction IDs**, customer assignments, or invalid dates.

---

### 4. Data Normalization (3NF-Aligned Structure)
Transformed the cleaned dataset into a fully normalized schema:
- `unit_t`: distinct units of measure
- `vendor_t`: all unique vendors extracted from purchases
- `item_type_t`: extracted item categories/types from product descriptions
- `product_t`: unique combinations of description, unit, vendor, and item type
- `location_t`: distinct store locations
- `inventory_t`: combination of `product_id` and `location_id` to model stock points
- `inventory_transaction_t`: captures all quantity movements tied to inventory items
- `purchase_t` & `sale_t`: transactional metadata (IDs, price, vendor/customer linkage, dates)
- `customer_t`: uniquely identified customers with consistent IDs

Each table was de-duplicated and assigned appropriate surrogate keys (`*_id` fields).

---

### 5. View Preparation
- Designed the structure for:
  - `purchase_transaction_summary_vw`
  - `sale_transaction_summary_vw`
  - `product_transaction_summary_vw`
- These will be defined within the database layer to simplify reporting across purchases and sales.

---

### 6. CSV Export
- All normalized tables exported as `.csv` files for seamless import into MySQL Workbench or any RDBMS.
- Files are named appropriately (e.g., `product_t.csv`, `inventory_transaction_t.csv`, etc.).

---

### Deliverables
- Fully cleaned and normalized transactional data
- 10+ relationally structured CSVs, ready for import
- View design documentation
- Manual correction notes (as code cells and in-line comments)

---

### Notes
- Inventory movement tracking is now possible via the normalized `inventory_transaction_t`, separating `SALE` and `PURCHASE` types.
- Manual adjustments were documented and carefully isolated to minimize data integrity issues.
- All transformations were conducted within a **reproducible notebook** to ensure transparency and repeatability.